##### Treinamento de usuário logado

In [ ]:
import pandas as pd
import numpy as np

##### Carregando Parquet

In [ ]:
user_csv_path = "../artifacts/parquets"
user_target_csv_file = "merged_data.parquet"

articles_csv_path = "../artifacts/parquets"
articles_target_csv_file = "articles_merged_data.parquet"

In [ ]:
users_df = pd.read_parquet(f"{user_csv_path}/{user_target_csv_file}")
noticias_df = pd.read_parquet(f"{articles_csv_path}/{articles_target_csv_file}")

##### Selecionando usuários logados

In [ ]:
logged_users_df = users_df[users_df['userType'] == 'Logged']

##### Pre processamento notícias

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
import string
from nltk.corpus import stopwords

nltk.data.clear_cache()
# nltk_data_path = "/home/heijimor/Desktop/fiap/datathon/train_service/artifacts/download/"  # ou outro caminho, se necessário
nltk.data.path.append(nltk_data_path)

# nltk.download('all', download_dir=nltk_data_path)
stop_words = set(stopwords.words('portuguese'))

In [ ]:
def preprocess_text(text):
    text = text.lower()
    text = ''.join([char for char in text if char not in string.punctuation])
    words = nltk.word_tokenize(text)
    return ' '.join([word for word in words if word not in stop_words])

noticias_df['processed_body'] = noticias_df['body'].apply(preprocess_text)
print(noticias_df[['title', 'processed_body']].head())

In [ ]:
def convert_to_numeric(clicks_str):
    clicks = list(map(int, clicks_str.split(',')))
    return sum(clicks)

logged_users_df['numberOfClicksHistory'] = logged_users_df['numberOfClicksHistory'].apply(convert_to_numeric)
logged_users_df['timeOnPageHistory'] = logged_users_df['timeOnPageHistory'].apply(convert_to_numeric)
logged_users_df['scrollPercentageHistory'] = logged_users_df['scrollPercentageHistory'].apply(
    lambda x: sum(map(float, x.split(','))) if isinstance(x, str) else 0
)
logged_users_df['pageVisitsCountHistory'] = logged_users_df['pageVisitsCountHistory'].apply(convert_to_numeric)

# # Normalizar os dados de comportamento do usuário e calcular o índice de engajamento
logged_users_df['engagement'] = (
    logged_users_df['numberOfClicksHistory'] * 0.2 + # Peso para cliques
    logged_users_df['timeOnPageHistory'] * 0.3 +      # Peso para tempo na página
    logged_users_df['scrollPercentageHistory'] * 0.2 + # Peso para scroll
    logged_users_df['pageVisitsCountHistory'] * 0.3    # Peso para visitas
)

# # Verificar o novo índice de engajamento
print(logged_users_df[['userId', 'engagement']].head())

##### Passo 2: Similaridade de conteúdo (TF-IDF)


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

stop_words_pt = stopwords.words('portuguese')

# Usar o TF-IDF para calcular a similaridade entre os corpos das notícias
tfidf_vectorizer = TfidfVectorizer(
    max_df=0.95,  # Remove termos muito comuns
    min_df=1,  # Remove termos muito raros
    max_features=10000,  # Aumenta o limite de palavras únicas
    stop_words=stop_words_pt  # Aqui passamos a lista correta!
)
tfidf_matrix = tfidf_vectorizer.fit_transform(noticias_df['processed_body'])

In [ ]:
from sklearn.decomposition import TruncatedSVD

# Reduzindo a dimensionalidade da matriz TF-IDF
sample = 0.2
svd = TruncatedSVD(n_components=100)  # Ajuste o número de componentes conforme necessário
sampled_indices = np.random.choice(tfidf_matrix.shape[0], size=int(tfidf_matrix.shape[0] * sample), replace=False)  # Amostra 20% das linhas
sampled_tfidf_matrix = tfidf_matrix[:int(tfidf_matrix.shape[0] * sample)]
reduced_tfidf = svd.fit_transform(sampled_tfidf_matrix)

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation

lda = LatentDirichletAllocation(n_components=5, random_state=42)
lda.fit(sampled_tfidf_matrix)
lda_topics = lda.transform(sampled_tfidf_matrix)


In [121]:
noticias_df['category'].fillna(0.0, inplace=True)
noticias_df.loc[sampled_indices, 'category'] = np.argmax(lda_topics, axis=1)
print(noticias_df.loc[sampled_indices, 'category'].head())


140780    1.0
160628    2.0
248329    0.0
40484     0.0
101704    0.0
Name: category, dtype: float64


##### Exibindo as 10 palavras mais relevantes para cada tópico

In [ ]:
n_words = 10
for topic_idx, topic in enumerate(lda.components_):
    print(f"Topic {topic_idx}:")
    print([tfidf_vectorizer.get_feature_names_out()[i] for i in topic.argsort()[:-n_words - 1:-1]])

##### Calcular a similaridade entre as notícias usando a matriz TF-IDF

In [ ]:
from sklearn.neighbors import NearestNeighbors

knn = NearestNeighbors(n_neighbors=10, metric='cosine')
knn.fit(reduced_tfidf)
distances, indices = knn.kneighbors(reduced_tfidf)

In [ ]:
import pickle

artifacts_models_path = "../artifacts/models"

with open(f"{artifacts_models_path}/knn_model.pkl", "wb") as f:
  pickle.dump(knn, f)

##### Ajuste da Similaridade com o Perfil do Usuário

In [ ]:
def ajusta_recomendacao(user_profile, similarity_matrix):
    # O perfil do usuário pode ser uma combinação de suas preferências (ex: mais tempo em esportes, etc.)
    # Por exemplo, se o usuário tem maior engajamento com esportes, aumenta a similaridade das notícias de esportes
    ajusted_similarity = similarity_matrix * user_profile
    return ajusted_similarity

##### Criar a função de recomendação

In [147]:
def recommend_news_knn(user_id, top_n=5):
    # print(noticias_df.loc[sampled_indices, 'category'].head())
    # Obter o comportamento do usuário logado
    user_data = logged_users_df[logged_users_df['userId'] == user_id]
    
    if user_data.empty:
        print("Usuário não encontrado.")
        return

    user_engagement = user_data['engagement'].values[0] # Encontrar o conteúdo mais relevante com base no engajamento
    user_history = user_data['history'].values[0] # Determinar as categorias mais consumidas pelo usuário
    # ISSO PRECISA SER PRE PROCESSADO DEPOIS
    user_history = user_history.split(", ")

    if not user_history:
        print("Nenhum histórico encontrado para o usuário.")
        return None

    sampled_history = [str(article) for article in user_history if str(article) in noticias_df['page'].astype(str).values]
    print(sampled_history)
    # Obter categorias baseadas nos artigos do histórico do usuário
    sample_news_pages = noticias_df.loc[sampled_indices, 'page'].astype(str)
    sample_news_pages = sample_news_pages.str.strip()

    print(sample_news_pages)
    # Verificar se as páginas amostradas estão na lista de histórico do usuário
    sample_page_id = sample_news_pages.isin(map(str, sampled_history))
    print(sample_page_id)
    # print(sample_page_id)
    print(sample_page_id.sum())  # Verificar quantos valores são True
    # print(noticias_df['category'].isnull().sum())  # Verificar se há valores nulos
    # # print(noticias_df['category'].head(2))

In [148]:
recommended_news_knn = recommend_news_knn(user_id='97e1439d485b0630e12818d3df84ff67d08475ef6ebeb08e5354779ad6a30fdb', top_n=6)
# print(recommended_news_knn)

['385044ad-3876-4188-83fa-f560435c1a9c', '2f754502-7014-488b-88c7-d41328d5e80d']
140780    c8a69d1a-206f-4ae1-84f0-e237832d93be
160628    365df623-85b5-4829-820c-3467930abfaa
248329    87179350-0b63-4b94-9b9d-b90b6bbef042
40484     94595cd8-ee10-49db-890a-22f15e9c6248
101704    8e101739-3f34-4af0-9b71-fb3e35d94e56
                          ...                 
252243    67c6ee47-7f2b-489f-9376-25ce2bb9adab
176825    81be96bd-9d7f-4033-a329-aa4cc6a540bb
57003     8e866c58-dd0b-4ee7-b7d6-4b5d8f7464cc
228911    d3eafc57-f82e-4c1d-bf59-03e0948092e4
146105    0861cdab-70b2-4605-9598-b139d2b81825
Name: page, Length: 51120, dtype: object
140780    False
160628    False
248329    False
40484     False
101704    False
          ...  
252243    False
176825    False
57003     False
228911    False
146105    False
Name: page, Length: 51120, dtype: bool
0


##### Avaliação e Ajuste Fino

In [ ]:
# Vamos calcular uma métrica simples de precisão para avaliar o modelo
# Assumindo que temos uma coluna 'liked_news' no dataframe 'logged_users_df' que indica quais notícias o usuário curtiu

# Simular a coluna 'liked_news' para avaliação (deve ser substituída por dados reais)
logged_users_df['liked_news'] = logged_users_df['userId'].apply(lambda x: noticias_df['title'].sample(1).values[0])

##### Avaliar a precisão

In [ ]:
def evaluate_recommendations(user_id, top_n=5):
    recommended_news = recommend_news_knn(user_id, top_n)
    print(f"recommended_news: {recommended_news}")
    if recommended_news is not None:
        liked_news = logged_users_df[logged_users_df['userId'] == user_id]['liked_news'].values[0]
        print(f"liked_news: {liked_news}")
        # Verificar se alguma das notícias recomendadas é a mesma que o usuário gostou
        hits = recommended_news[recommended_news['title'].isin([liked_news])].shape[0]
        print(f"hits: {hits}")
        precision = hits / top_n  # Precisão (fração de acertos nas top_n notícias recomendadas)
        print(f"precision: {precision}")
        return precision
    else:
        return 0

precision = evaluate_recommendations(user_id='97e1439d485b0630e12818d3df84ff67d08475ef6ebeb08e5354779ad6a30fdb', top_n=6)
print(f'Precisão da recomendação: {precision * 100}%')